# Setup

In [0]:
from tqdm import tqdm
import PIL as pil
from PIL import Image
from PIL.Image import fromarray
import os
import csv
import google.protobuf
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Flatten, Activation, Dropout, Conv2D, MaxPooling2D, Input, Conv2DTranspose, concatenate, UpSampling2D
from IPython.display import Image, display
import sys
import threading
import pickle
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Activation
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects
from tempfile import TemporaryFile
from random import shuffle

print(tf.VERSION)
print(tf.keras.__version__)

from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

# Utility functions
Here functions for datainteraction are defined


**Picture information saving class**

In [0]:
class pixel_array:
 __coordinate=0
 __length=0
 
 def __init__(self, coordinate, length):
  self.__coordinate = int (coordinate)
  self.__length = int (length)
 
 def setCoordinate(self, xCoordinate):
    self.__coordinate = coordinate
 
 def setLength(self, yCoordinate):
    self.__length = length
   
 def get_coordinate(self):
    return int(self.__coordinate)
   
 def get_length(self):
    return int(self.__length)

**Plot the history**

In [0]:
def plotHistory(iou_array, val_iou_array):
    
  fig = plt.figure(figsize=(12, 6))
  ax = fig.add_subplot(1, 2, 1)
  ax.plot(iou_array,'r-x', label="Train IOU")
  ax.legend()
  ax.set_title('iou accuracy')
  ax.grid(True)
  ax.plot(val_iou_array,'b-x', label="Validation IOU")
  ax.legend()
  ax.grid(True)

**Get labels from file as dictionary of pixel_array with image name as key**

In [0]:
def readMarkedPixels(path):
   num_lines = sum(1 for line in open(path))
   num_lines = num_lines-1
   with open(path, mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    line_count = 0
    data = {}
    for row in tqdm(csv_reader, total=num_lines, unit="rows"):

     pixelData = row['EncodedPixels'].split(' ')
     counter = 0
     pixels = []

     while counter < len(pixelData)-1:
      pixels.append(pixel_array(pixelData[counter], pixelData[counter+1]))
      counter += 2
     image_id = row['ImageId']
     try:
      data[image_id].extend(pixels)
     except Exception as e:
      data[image_id] = pixels
      
     if len(data[image_id])<10:
      data[image_id] = []
    
    return data

**Batch separation function**

In [0]:
def getBatchList(batch_size, index, all_list):
    return all_list[index*batch_size:(index+1)*batch_size]

# Preprocessing
The following chapter handels the creation of Masks from the given data.

**Creating the Mask Thread**

In [0]:
class createLabelsThread(threading.Thread):

    def __init__(self, marked_pixels, work_list, target_path, group=None, target=None, name=None,
                 args=(), kwargs=None, verbose=None):
        threading.Thread.__init__(self, group=group, target=target, name=name)
        self.args = args
        self.kwargs = kwargs
        self.work_list = work_list
        self.marked_pixels = marked_pixels
        self.target_path = target_path
        return

    def run(self):
        for key in self.work_list:
          temp_label = np.zeros((768**2))
          for pixel_arrays in self.marked_pixels[key]:
            for i in range(pixel_arrays.get_length()):
              try:
                temp_label[pixel_arrays.get_coordinate()-1 + i] = 1
              except:
                print("except in : " + str(key))
          temp_label.tofile(self.target_path + key + ".data")
        return

**Execute Mask creation**

Training Masks


---



In [0]:
# Get label data
marked_train_pixels = readMarkedPixels('./train_label.csv')

# Split data into containing and not containing ships
img_cont_ship = []
img_no_cont_ship = []
counter = 0
contain_ships = 0
for key in marked_train_pixels:
  counter +=1
  if len(marked_train_pixels[key]) != 0:
    contain_ships += 1
    img_cont_ship.append(key)
  else:
    img_no_cont_ship.append(key)
  
print("number of ships: " + str(contain_ships))
print("number of non_ships: " + str(counter))
print("contain ships: " + str(contain_ships * 100/counter) + "%")

In [0]:
# create train labels with Multithreading

marked_train_pixels = readMarkedPixels('./train_label.csv')

threads = []

batch_size = 1250
threads.append(createLabelsThread(marked_pixels = marked_train_pixels, work_list = getBatchList(batch_size, 0, img_cont_ship), target_path = "./train_label_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_train_pixels, work_list = getBatchList(batch_size, 1, img_cont_ship), target_path = "./train_label_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_train_pixels, work_list = getBatchList(batch_size, 2, img_cont_ship), target_path = "./train_label_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_train_pixels, work_list = getBatchList(batch_size, 3, img_cont_ship), target_path = "./train_label_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_train_pixels, work_list = getBatchList(batch_size, 4, img_cont_ship), target_path = "./train_label_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_train_pixels, work_list = getBatchList(batch_size, 5, img_cont_ship), target_path = "./train_label_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_train_pixels, work_list = getBatchList(batch_size, 6, img_cont_ship), target_path = "./train_label_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_train_pixels, work_list = getBatchList(batch_size, 7, img_cont_ship), target_path = "./train_label_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_train_pixels, work_list = getBatchList(batch_size, 8, img_cont_ship), target_path = "./train_label_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_train_pixels, work_list = getBatchList(batch_size, 9, img_cont_ship), target_path = "./train_label_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_train_pixels, work_list = getBatchList(batch_size, 10, img_cont_ship), target_path = "./train_label_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_train_pixels, work_list = getBatchList(batch_size, 11, img_cont_ship), target_path = "./train_label_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_train_pixels, work_list = getBatchList(batch_size, 12, img_cont_ship), target_path = "./train_label_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_train_pixels, work_list = getBatchList(batch_size, 13, img_cont_ship), target_path = "./train_label_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_train_pixels, work_list = getBatchList(batch_size, 14, img_cont_ship), target_path = "./train_label_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_train_pixels, work_list = getBatchList(batch_size, 15, img_cont_ship), target_path = "./train_label_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_train_pixels, work_list = getBatchList(batch_size, 16, img_cont_ship), target_path = "./train_label_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_train_pixels, work_list = getBatchList(batch_size, 17, img_cont_ship), target_path = "./train_label_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_train_pixels, work_list = getBatchList(batch_size, 18, img_cont_ship), target_path = "./train_label_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_train_pixels, work_list = getBatchList(batch_size, 19, img_cont_ship), target_path = "./train_label_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_train_pixels, work_list = getBatchList(batch_size, 0, img_no_cont_ship), target_path = "./train_label_no_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_train_pixels, work_list = getBatchList(batch_size, 1, img_no_cont_ship), target_path = "./train_label_no_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_train_pixels, work_list = getBatchList(batch_size, 2, img_no_cont_ship), target_path = "./train_label_no_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_train_pixels, work_list = getBatchList(batch_size, 3, img_no_cont_ship), target_path = "./train_label_no_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_train_pixels, work_list = getBatchList(batch_size, 4, img_no_cont_ship), target_path = "./train_label_no_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_train_pixels, work_list = getBatchList(batch_size, 5, img_no_cont_ship), target_path = "./train_label_no_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_train_pixels, work_list = getBatchList(batch_size, 6, img_no_cont_ship), target_path = "./train_label_no_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_train_pixels, work_list = getBatchList(batch_size, 7, img_no_cont_ship), target_path = "./train_label_no_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_train_pixels, work_list = getBatchList(batch_size, 8, img_no_cont_ship), target_path = "./train_label_no_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_train_pixels, work_list = getBatchList(batch_size, 9, img_no_cont_ship), target_path = "./train_label_no_ship/"))

for t in threads:
  t.start()
  
print("waiting for finish...")

for t in threads: 
  t.join()
  
print("finished!")


Test Masks


---



In [0]:
# Get Label data
marked_test_pixels = readMarkedPixels('./test_label.csv')


# Split data into containing and not containing ships
test_img_cont_ship = []
test_img_no_cont_ship = []
test_counter = 0
test_contain_ships = 0
for key in marked_test_pixels:
  test_counter +=1
  if len(marked_test_pixels[key]) != 0:
    test_contain_ships += 1
    test_img_cont_ship.append(key)
  else:
    test_img_no_cont_ship.append(key)
  
print("number of ships: " + str(test_contain_ships))
print("number of non_ships: " + str(len(test_img_no_cont_ship)))
print("contain ships: " + str(test_contain_ships * 100/test_counter) + "%")

In [0]:
# Create test labels with Multithreading## create labels

marked_test_pixels = readMarkedPixels('./test_label.csv')

threads = []

batch_size = 1262
threads.append(createLabelsThread(marked_pixels = marked_test_pixels, work_list = getBatchList(batch_size, 0, test_img_cont_ship), target_path = "./test_label_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_test_pixels, work_list = getBatchList(batch_size, 1, test_img_cont_ship), target_path = "./test_label_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_test_pixels, work_list = getBatchList(batch_size, 2, test_img_cont_ship), target_path = "./test_label_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_test_pixels, work_list = getBatchList(batch_size, 3, test_img_cont_ship), target_path = "./test_label_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_test_pixels, work_list = getBatchList(batch_size, 4, test_img_cont_ship), target_path = "./test_label_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_test_pixels, work_list = getBatchList(batch_size, 5, test_img_cont_ship), target_path = "./test_label_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_test_pixels, work_list = getBatchList(batch_size, 6, test_img_cont_ship), target_path = "./test_label_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_test_pixels, work_list = getBatchList(batch_size, 7, test_img_cont_ship), target_path = "./test_label_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_test_pixels, work_list = getBatchList(batch_size, 8, test_img_cont_ship), target_path = "./test_label_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_test_pixels, work_list = getBatchList(batch_size, 9, test_img_cont_ship), target_path = "./test_label_ship/"))


batch_size = 3792
threads.append(createLabelsThread(marked_pixels = marked_test_pixels, work_list = getBatchList(batch_size, 0, test_img_no_cont_ship), target_path = "./test_label_no_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_test_pixels, work_list = getBatchList(batch_size, 1, test_img_no_cont_ship), target_path = "./test_label_no_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_test_pixels, work_list = getBatchList(batch_size, 2, test_img_no_cont_ship), target_path = "./test_label_no_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_test_pixels, work_list = getBatchList(batch_size, 3, test_img_no_cont_ship), target_path = "./test_label_no_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_test_pixels, work_list = getBatchList(batch_size, 4, test_img_no_cont_ship), target_path = "./test_label_no_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_test_pixels, work_list = getBatchList(batch_size, 5, test_img_no_cont_ship), target_path = "./test_label_no_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_test_pixels, work_list = getBatchList(batch_size, 6, test_img_no_cont_ship), target_path = "./test_label_no_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_test_pixels, work_list = getBatchList(batch_size, 7, test_img_no_cont_ship), target_path = "./test_label_no_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_test_pixels, work_list = getBatchList(batch_size, 8, test_img_no_cont_ship), target_path = "./test_label_no_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_test_pixels, work_list = getBatchList(batch_size, 9, test_img_no_cont_ship), target_path = "./test_label_no_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_test_pixels, work_list = getBatchList(batch_size, 10, test_img_no_cont_ship), target_path = "./test_label_no_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_test_pixels, work_list = getBatchList(batch_size, 11, test_img_no_cont_ship), target_path = "./test_label_no_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_test_pixels, work_list = getBatchList(batch_size, 12, test_img_no_cont_ship), target_path = "./test_label_no_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_test_pixels, work_list = getBatchList(batch_size, 13, test_img_no_cont_ship), target_path = "./test_label_no_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_test_pixels, work_list = getBatchList(batch_size, 14, test_img_no_cont_ship), target_path = "./test_label_no_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_test_pixels, work_list = getBatchList(batch_size, 15, test_img_no_cont_ship), target_path = "./test_label_no_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_test_pixels, work_list = getBatchList(batch_size, 16, test_img_no_cont_ship), target_path = "./test_label_no_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_test_pixels, work_list = getBatchList(batch_size, 17, test_img_no_cont_ship), target_path = "./test_label_no_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_test_pixels, work_list = getBatchList(batch_size, 18, test_img_no_cont_ship), target_path = "./test_label_no_ship/"))
threads.append(createLabelsThread(marked_pixels = marked_test_pixels, work_list = getBatchList(batch_size, 19, test_img_no_cont_ship), target_path = "./test_label_no_ship/"))

for t in threads:
  t.start()
  
print("waiting for finish...")

for t in threads: 
  t.join()
  
print("finished!")


# Getting to know the data

**Visualisation of Original image and Mask**

In [0]:
display(Image("./train_data/6384c3e78.jpg", width=300, height=300))
plt.imshow(np.reshape(np.fromfile("./train_label/00aa60389.jpg.data")*255, (768, 768)).T, cmap="gray")

# Training

** Define loss function and metrics**

In [0]:
# Iou metric
def iou(y_true, y_pred):
  thresh = 0
  y_true_f = K.flatten(y_true)
  y_pred_f = K.flatten(y_pred)
  #calculate true positive, false negative and false positive values
  true_positive = K.sum(y_true_f * y_pred_f)
  false_negative = K.sum(y_true_f) - true_positive
  false_positive = K.sum(y_pred_f) - true_positive
  beta = 2
  return K.cast(((1 + beta^2) * true_positive + thresh)/(((1 + beta^2) * true_positive) + (beta^2)*false_negative + false_positive + thresh), 'float32')

# Back_iou metric
def back_iou(y_true, y_pred):
   return iou(1-y_true, 1-y_pred)
  
# Loss function
def iou_loss(y_true, y_pred):
  return K.cast(1, 'float32') - iou(y_true, y_pred)

**Data generator**

Providing the data as a pipeline.

In [0]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, read_path, batch_size=32, dim=(768,768), n_channels=3,
                 n_classes=2, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.read_path = read_path
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, 768, 768, self.n_channels))
        y = np.empty((self.batch_size, 768, 768))

        label_dir = self.read_path.split("_")[0] + "_label_no_ship/"#"_label_ship/"
        
        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store data
            try:
                X[i,] = np.asarray(pil.Image.open(self.read_path + ID))/255

            # Store label
                y[i] = np.reshape(np.fromfile(label_dir + ID + ".data"), (768, 768, 1)).T
            except Exception as e:
              print("error in file: " + str(ID))
              print(str(e))

        return X, y

Creating validation and and train datasets

In [0]:
params = {'batch_size': 3,
          'dim': (768,768),
          'n_channels': 3,
          'n_classes': 2,
          'shuffle': True}

# Read filenames
complete_data_list = os.listdir("./train_label_ship/")
for i, string in enumerate(complete_data_list):
    complete_data_list[i] = string[:-5]

shuffle(complete_data_list)

print(str(complete_data_list[0]))
print(len(complete_data_list))
# Split dataset
data1 = complete_data_list[0:6249]
data2 = complete_data_list[6250:12499]
data3 = complete_data_list[12500:18749]
data4 = complete_data_list[18750:24999]

# Create sets
training_data_1 = data1
training_data_1.extend(data2)
training_data_1.extend(data3)
validation_data_1 = data4

training_data_2 = data1
training_data_2.extend(data2)
training_data_2.extend(data4)
validation_data_2 = data3

training_data_3 = data1
training_data_3.extend(data3)
training_data_3.extend(data4)
validation_data_3 = data2

training_data_4 = data2
training_data_4.extend(data3)
training_data_4.extend(data4)
validation_data_4 = data1

# Create data generators
training_generator1 = DataGenerator(training_data_1, "./train_data/", **params)
validation_generator1 = DataGenerator(validation_data_1, "./train_data/", **params)

training_generator2 = DataGenerator(training_data_2, "./train_data/", **params)
validation_generator2 = DataGenerator(validation_data_2, "./train_data/", **params)

training_generator3 = DataGenerator(training_data_3, "./train_data/", **params)
validation_generator3 = DataGenerator(validation_data_3, "./train_data/", **params)

training_generator4 = DataGenerator(training_data_4, "./train_data/", **params)
validation_generator4 = DataGenerator(validation_data_4, "./train_data/", **params)

**Define reusable learning function**

In [0]:
def learn(save_model_path, training_generator, validation_generator, model):
  # Compile
  model.compile(optimizer=Adam(lr=1e-7), loss = iou_loss, metrics=[iou])
  
  # Create callbacks
  checkpoint = ModelCheckpoint(save_model_path, monitor='val_iou', verbose=1, 
                               save_best_only=True, mode='max', save_weights_only = True)

  early = EarlyStopping(monitor="val_iou", 
                        mode="max", 
                        patience=2)

  callbacks_list = [checkpoint, early]


  # Train model on dataset
  history = model.fit_generator(generator=training_generator,
                      validation_data=validation_generator,
                      use_multiprocessing=True,
                      epochs=50,
                      workers=30,
                      callbacks=callbacks_list,
                      verbose=1)

  # Save history
  print("\n" + save_model_path + " history:")
    
  iou_val_history = history.history["val_iou"]
  numpy_val_iou_history = np.array(iou_val_history)
  np.savetxt(save_model_path[:-3]+"_val_iou_history.txt", numpy_val_iou_history, delimiter=",")
 
  iou_history = history.history["iou"]
  numpy_iou_history = np.array(iou_history)
  np.savetxt(save_model_path[:-3]+"_iou_history.txt", numpy_iou_history, delimiter=",")


**Model Transpose Convolution 1**


---






In [0]:
def buildModel(empty_model):

    inputs = Input((768, 768, 3))

    c0 = Conv2D(8, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (inputs)
    c0 = Dropout(0.1) (c0)
    c0 = Conv2D(8, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c0)
    p0 = MaxPooling2D((2, 2)) (c0)

    c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p0)
    c1 = Dropout(0.1) (c1)
    c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
    c2 = Dropout(0.1) (c2)
    c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
    c3 = Dropout(0.2) (c3)
    c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
    c4 = Dropout(0.2) (c4)
    c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

    c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
    c5 = Dropout(0.3) (c5)
    c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
    c6 = Dropout(0.2) (c6)
    c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
    c7 = Dropout(0.2) (c7)
    c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
    c8 = Dropout(0.1) (c8)
    c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

    u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
    c9 = Dropout(0.1) (c9)
    c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

    u10 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c9)
    u10 = concatenate([u10, c0], axis=3)
    c10 = Conv2D(8, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u10)
    c10 = Dropout(0.1) (c10)
    c10 = Conv2D(8, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c10)

    outputs =Conv2D(1, (1, 1), activation='sigmoid') (c10)

    empty_model = Model(inputs=[inputs], outputs=[outputs])
    #tc_u_net_model.summary()
    return empty_model
    



In [0]:
# Train Model 1
val1_tc_u_net_model_N1 = None
val2_tc_u_net_model_N1 = None
val3_tc_u_net_model_N1 = None
val4_tc_u_net_model_N1 = None
learn("./val1_tc_u_net_model_N1.h5", training_generator1, validation_generator1, buildModel(val1_tc_u_net_model_N1))
learn("./val2_tc_u_net_model_N1.h5", training_generator2, validation_generator2, buildModel(val2_tc_u_net_model_N1))
learn("./val3_tc_u_net_model_N1.h5", training_generator3, validation_generator3, buildModel(val3_tc_u_net_model_N1))
learn("./val4_tc_u_net_model_N1.h5", training_generator4, validation_generator4, buildModel(val4_tc_u_net_model_N1))

**Model Bilinear Interpolation**


---






In [0]:
def buildModel2(empty_model):
    inputs = Input((768, 768, 3))

    c0 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (inputs)
    c0 = Dropout(0.1) (c0)
    c0 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c0)
    p0 = MaxPooling2D((2, 2)) (c0)

    c1 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p0)
    c1 = Dropout(0.1) (c1)
    c1 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
    c2 = Dropout(0.2) (c2)
    c2 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
    c3 = Dropout(0.2) (c3)
    c3 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)

    u1 = UpSampling2D(size=(2, 2), data_format=None) (c3)
    c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u1)
    c4 = Dropout(0.2) (c4)
    c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)

    u2 = UpSampling2D(size=(2, 2), data_format=None) (c4)
    c5 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u2)
    c5 = Dropout(0.2) (c5)
    c5 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

    u3 = UpSampling2D(size=(2, 2), data_format=None) (c5)
    c6 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u3)
    c6 = Dropout(0.2) (c6)
    c6 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

    outputs = Conv2D(1, (1, 1), activation='sigmoid') (c6)

    empty_model = Model(inputs=[inputs], outputs=[outputs])
    return empty_model
    #bi_u_net_model.summary()


In [0]:
# Train model 2
val1_tc_u_net_model_N2 = None
val2_tc_u_net_model_N2 = None
val3_tc_u_net_model_N2 = None
val4_tc_u_net_model_N2 = None
learn("./val1_tc_u_net_model_N2.h5", training_generator2, validation_generator2, buildModel2(val1_tc_u_net_model_N2))
learn("./val2_tc_u_net_model_N2.h5", training_generator2, validation_generator2, buildModel2(val2_tc_u_net_model_N2))
learn("./val3_tc_u_net_model_N2.h5", training_generator3, validation_generator3, buildModel2(val3_tc_u_net_model_N2))
learn("./val4_tc_u_net_model_N2.h5", training_generator4, validation_generator4, buildModel2(val4_tc_u_net_model_N2))

**Model Transposed Convolution 2**


---






In [0]:
def buildModel3(empty_model):

    inputs = Input((768, 768, 3))

    c0 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (inputs)
    c0 = Dropout(0.1) (c0)
    c0 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c0)
    p0 = MaxPooling2D((2, 2)) (c0)

    c1 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p0)
    c1 = Dropout(0.1) (c1)
    c1 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
    c2 = Dropout(0.1) (c2)
    c2 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = Conv2D(512, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
    c3 = Dropout(0.2) (c3)
    c3 = Conv2D(512, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = Conv2D(1024, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)

    u5 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same') (c4)
    u5 = concatenate([u5, c3])
    c5 = Conv2D(512, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u5)
    c5 = Dropout(0.2) (c5)
    c5 = Conv2D(512, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)

    u6 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c2])
    c6 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
    c6 = Dropout(0.2) (c6)
    c6 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)

    u7 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c1])
    c7 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
    c7 = Dropout(0.1) (c7)
    c7 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

    u8 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c0])
    c8 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
    c8 = Dropout(0.1) (c8)
    c8 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

    outputs = Conv2D(1, (1, 1), activation='sigmoid') (c8)

    empty_model = Model(inputs=[inputs], outputs=[outputs])
    empty_model.summary()
    return empty_model

In [0]:
# Train model 3
val1_tc_u_net_model_N3 = None
val2_tc_u_net_model_N3 = None
val3_tc_u_net_model_N3 = None
val4_tc_u_net_model_N3 = None
learn("./val1_tc_u_net_model_N3.h5", training_generator2, validation_generator2, buildModel3(val1_tc_u_net_model_N3))
learn("./val2_tc_u_net_model_N3.h5", training_generator2, validation_generator2, buildModel3(val2_tc_u_net_model_N3))
learn("./val3_tc_u_net_model_N3.h5", training_generator3, validation_generator3, buildModel3(val3_tc_u_net_model_N3))
learn("./val4_tc_u_net_model_N3.h5", training_generator4, validation_generator4, buildModel3(val4_tc_u_net_model_N3))

# Evaluation

In [0]:
# Define back_iou evaluation function
def back_iou_eval(model, weights, generator, name):
    model.compile(optimizer=Adam(lr=1e-4), loss = iou_loss, metrics=[back_iou])
    model.load_weights(weights)

    scores = model.evaluate_generator(generator=generator, verbose=1, workers=30, use_multiprocessing=True)

    print(name)
    print(scores[1])
    print("---------------")

In [0]:
# Define iou evaluation function
def iou_eval(model, weights, generator, name):
    model.compile(optimizer=Adam(lr=1e-4), loss = iou_loss, metrics=[iou])
    model.load_weights(weights)

    scores = model.evaluate_generator(generator=generator, verbose=1, workers=30, use_multiprocessing=True)

    print(name)
    print(scores[1])
    print("---------------")

In [0]:
# Creating ship evaluation generators
params = {'batch_size': 8,
          'dim': (768,768),
          'n_channels': 3,
          'n_classes': 2,
          'shuffle': True}

complete_data_list = os.listdir("./test_label_ship/")
for i, string in enumerate(complete_data_list):
    complete_data_list[i] = string[:-5]
    
print(str(complete_data_list[0]))
print(len(complete_data_list))


test_ship_generator = DataGenerator(complete_data_list, "./test_data/", **params)

In [0]:
# Creating all evaluation generators
params = {'batch_size': 3,
          'dim': (768,768),
          'n_channels': 3,
          'n_classes': 2,
          'shuffle': True}

ship_data_list = os.listdir("./test_label_ship/")
for i, string in enumerate(ship_data_list):
    ship_data_list[i] = string[:-5]
    
complete_data_list = os.listdir("./test_label_no_ship/")
for i, string in enumerate(complete_data_list):
    complete_data_list[i] = string[:-5]
    
complete_data_list.extend(ship_data_list)
shuffle(complete_data_list)
complete_data_list = complete_data_list[0:int(np.floor(len(complete_data_list) / 4))]

print(str(complete_data_list[0]))
print(len(complete_data_list))


test_all_generator = DataGenerator(complete_data_list, "./test_data/", **params)

**Evaluate IOU on Test**


---

In [0]:
# Evaluate transpose convolution model 1 IOU on Test
model = None
iou_eval(buildModel(model), "./val1_tc_u_net_model_N1.h5", test_ship_generator, "back_iou_val1_N1")
model = None
iou_eval(buildModel(model), "./val2_tc_u_net_model_N1.h5", test_ship_generator, "back_iou_val2_N1")
model = None
iou_eval(buildModel(model), "./val3_tc_u_net_model_N1.h5", test_ship_generator, "back_iou_val3_N1")
model = None
iou_eval(buildModel(model), "./val4_tc_u_net_model_N1.h5", test_ship_generator, "back_iou_val4_N1")
#--------------------------------------------------------------------------------------------------------------
# Evaluate Bilinear Interpolation model IOU on Test
model = None
iou_eval(buildModel2(model), "./val1_tc_u_net_model_N2.h5", test_ship_generator, "back_iou_val1_N2")
model = None
iou_eval(buildModel2(model), "./val2_tc_u_net_model_N2.h5", test_ship_generator, "back_iou_val2_N2")
model = None
iou_eval(buildModel2(model), "./val3_tc_u_net_model_N2.h5", test_ship_generator, "back_iou_val3_N2")
model = None
iou_eval(buildModel2(model), "./val4_tc_u_net_model_N2.h5", test_ship_generator, "back_iou_val4_N2")
#--------------------------------------------------------------------------------------------------------------
# Evaluate transpose convolution model 2 IOU on Test
model = None
iou_eval(buildModel3(model), "./val1_tc_u_net_model_N3.h5", test_ship_generator, "back_iou_val1_N3")
model = None
iou_eval(buildModel3(model), "./val2_tc_u_net_model_N3.h5", test_ship_generator, "back_iou_val2_N3")
model = None
iou_eval(buildModel3(model), "./val3_tc_u_net_model_N3.h5", test_ship_generator, "back_iou_val3_N3")
model = None
iou_eval(buildModel3(model), "./val4_tc_u_net_model_N3.h5", test_ship_generator, "back_iou_val4_N3")

**Evaluate Back_IOU on Validation**


---

In [0]:
# Evaluate transpose convolution model 1 on Back-IoU
model = None
back_iou_eval(buildModel(model), "./val1_tc_u_net_model_N1.h5", validation_generator1, "back_iou_val1_N1")
model = None
back_iou_eval(buildModel(model), "./val2_tc_u_net_model_N1.h5", validation_generator2, "back_iou_val2_N1")
model = None
back_iou_eval(buildModel(model), "./val3_tc_u_net_model_N1.h5", validation_generator3, "back_iou_val3_N1")
model = None
back_iou_eval(buildModel(model), "./val4_tc_u_net_model_N1.h5", validation_generator3, "back_iou_val4_N1")

In [0]:
# Evaluate bilinear Interpolation model on Back-IoU
model = None
back_iou_eval(buildModel2(model), "./val1_tc_u_net_model_N2.h5", validation_generator1, "back_iou_val1_N2")
model = None
back_iou_eval(buildModel2(model), "./val2_tc_u_net_model_N2.h5", validation_generator2, "back_iou_val2_N2")
model = None
back_iou_eval(buildModel2(model), "./val3_tc_u_net_model_N2.h5", validation_generator3, "back_iou_val3_N2")
model = None
back_iou_eval(buildModel2(model), "./val4_tc_u_net_model_N2.h5", validation_generator3, "back_iou_val4_N2")

In [0]:
# Evaluate transpose convolution model 2 on Back-IoU
model = None
back_iou_eval(buildModel3(model), "./val1_tc_u_net_model_N3.h5", validation_generator1, "back_iou_val1_N3")
model = None
back_iou_eval(buildModel3(model), "./val2_tc_u_net_model_N3.h5", validation_generator2, "back_iou_val2_N3")
model = None
back_iou_eval(buildModel3(model), "./val3_tc_u_net_model_N3.h5", validation_generator1, "back_iou_val3_N3")
model = None
back_iou_eval(buildModel3(model), "./val4_tc_u_net_model_N3.h5", validation_generator2, "back_iou_val4_N3")

**Evaluate Back_IOU on Test**


---

In [0]:
# Evaluate transpose convolution model 1 on Test
model = None
back_iou_eval(buildModel(model), "./val1_tc_u_net_model_N1.h5", test_all_generator, "back_iou_val1_N1")
model = None
back_iou_eval(buildModel(model), "./val2_tc_u_net_model_N1.h5", test_all_generator, "back_iou_val2_N1")
model = None
back_iou_eval(buildModel(model), "./val3_tc_u_net_model_N1.h5", test_all_generator, "back_iou_val3_N1")
model = None
back_iou_eval(buildModel(model), "./val4_tc_u_net_model_N1.h5", test_all_generator, "back_iou_val4_N1")
#--------------------------------------------------------------------------------------------------------------
# Evaluate Bilinear Interpolation model on Test
model = None
back_iou_eval(buildModel2(model), "./val1_tc_u_net_model_N2.h5", test_all_generator, "back_iou_val1_N2")
model = None
back_iou_eval(buildModel2(model), "./val2_tc_u_net_model_N2.h5", test_all_generator, "back_iou_val2_N2")
model = None
back_iou_eval(buildModel2(model), "./val3_tc_u_net_model_N2.h5", test_all_generator, "back_iou_val3_N2")
model = None
back_iou_eval(buildModel2(model), "./val4_tc_u_net_model_N2.h5", test_all_generator, "back_iou_val4_N2")
#--------------------------------------------------------------------------------------------------------------
# Evaluate transpose convolution model 2 on Test
model = None
back_iou_eval(buildModel3(model), "./val1_tc_u_net_model_N3.h5", test_all_generator, "back_iou_val1_N3")
model = None
back_iou_eval(buildModel3(model), "./val2_tc_u_net_model_N3.h5", test_all_generator, "back_iou_val2_N3")
model = None
iou_eval(buildModel3(model), "./val3_tc_u_net_model_N3.h5", test_all_generator, "back_iou_val3_N3")
model = None
back_iou_eval(buildModel3(model), "./val4_tc_u_net_model_N3.h5", test_all_generator, "back_iou_val4_N3")